In [1]:
include("../src/CMPBP.jl")
using .CMPBP
using LinearAlgebra

const CMPS{F,N} = Array{F,N} where {F,N};

In [2]:
function onebpiter_infinite_regular!(A::CMPS{F,5}, factor, prob;
    maxiter=100, tol=1e-12, maxit_ascent=100, tol_ascent=1e-12 , maxit_pow=1000, tol_pow=1e-12) where F<:Number
    q,d = size(A,1), size(A,4)
    size(A,2)==size(A,3)==q && size(A,5)==d || error("Wrong dimensions for A, got $(size(A))")

    E(x,y; q=2) = [i==x && j==y for i in 1:q, j in 1:q]

    Anew = rand(size(A)...)
    AA = zeros(F, q,q,q,(q*d)^2,(q*d)^2)
    for it in 1:maxiter
        for x in 1:q, y in 1:q, z in 1:q
            AA[x,y,z,:,:] = sum(prob(factor, y,x, x₁ᵗ,x₂ᵗ,z)*kron(E(x₁ᵗ,x₁ᵗ), E(x₂ᵗ,x₂ᵗ), I(d), I(d)) for x₁ᵗ in 1:q, x₂ᵗ in 1:q) +
                (x==y) * (sum(kron(I(q), E(x₂ᵗ,x₂ᵗ⁺¹;q), I(d), A[x₂ᵗ,x₂ᵗ⁺¹,x,:,:]) for x₂ᵗ in 1:q, x₂ᵗ⁺¹ in 1:q) +
                        sum(kron(E(x₁ᵗ,x₁ᵗ⁺¹; q), I(q), A[x₁ᵗ,x₁ᵗ⁺¹,x,:,:], I(d)) for x₁ᵗ in 1:q, x₁ᵗ⁺¹ in 1:q))
        end
        ascent!(Anew,AA; maxiter=maxit_ascent, tol=tol_ascent, maxiter_pow=maxit_pow, tol_pow=tol_pow)
        if norm(Anew - A) < tol
            return it, Anew
        end
        A .= Anew
    end
    return maxiter, Anew
end

onebpiter_infinite_regular! (generic function with 1 method)

In [36]:
q = 2
d = 5

A = rand(q,q,q,d,d)

k = 3
β = 0.5
J = 1.0
h = -0.2

potts2spin(x, i=0; q=2) = (x-1)/(q-1)*2 - 1

function prob_glauber(factor, y,x,x∂i...)
    β, J, h = factor.β, factor.J, factor.h
    localfield = β*(h + J*sum(potts2spin, x∂i))
    p = exp(potts2spin(y)*localfield) / (2*cosh(localfield))
    return y==x ? -p : p
end

struct GlauberFactor{F}
    β::F
    J::F
    h::F
end

function magnetization(A)
    p = marginals(A)
    m = sum(potts2spin(x)*p[x] for x in eachindex(p))
end


w = GlauberFactor(β, J, h)
;

In [72]:
onebpiter_infinite_regular!(A, w, prob_glauber; maxiter=1, tol=1e-12, maxit_ascent=1000,tol_ascent=1e-12, maxit_pow=1000, tol_pow=1e-12)

(1, [0.5260485540223543 0.41985015084130484; -0.12382163491675038 0.32511123825792065;;; 0.25537856944082726 -0.2607011070858535; 0.6098957682351343 0.02377794868932119;;;; 0.4112173472188414 -0.202545293109298; 0.5793627453193555 1.0887011216860198;;; 0.7713590786103898 0.17453876091315926; 0.7586645711728391 1.1109759511460708;;;; 0.5532992760702011 0.27306779608861886; 0.3118339544465116 0.5709880143779743;;; 0.6185094979391123 0.5826164574324343; 0.5394369130774731 1.0251131433531595;;;; 0.23651043371408487 0.6027656005405863; -0.006601345762435495 0.9931595046856218;;; -0.014303635867567497 0.12815556812154658; 0.6349484477359586 1.0334113837368046;;;; 0.44923146690284366 0.3027369988179776; -0.08123384567870533 0.7798335326656938;;; 0.40665658916685776 0.5249306074558158; 0.36250599890602075 0.9843099253829843;;;;; 0.18949060417023691 -0.5081064438358844; 0.3953615660568585 0.4940356827571062;;; 0.42092563868843297 -0.3325714176940085; 0.16735038468331645 0.02028313820637955;;;; 

In [73]:
marginals(A)

2-element Vector{ComplexF64}:
 0.06849013356781283 + 6.227283836352544e-17im
  0.9315098664321871 - 6.227283836352542e-17im

In [74]:
magnetization(A)

0.8630197328643743 - 1.2454567672705087e-16im

In [32]:
using MatrixProductBP.Models
equilibrium_observables(RandomRegular(3), J; β, h).m

-0.7084187895333882